#### Parameters
- The set of engines that needs maintenance is $M = {1,2,...j}$
- The planning horizon is $T = {1,2,...,t}$
- The available groups are $G = {1,2,..g}$
- The number of working days of  team g on engine j is $\mu_{g,j}$
- Let $P_{g,j,t}$ be the penalty if team $g$ starts working on engine $j$ from day $t$
- Let $Q_j$ be the cost of engine $j$ if the maintenance is not done during the planning horizon $T$ 

#### Object function 
$min$  $Z = \sum_{j=1}^{M} \sum_{g=1}^{G} \sum_{t=1}^{T} P_{g,j,t}* X_{g,j,t} + \sum_{j=1}^{M} (( 1 - \sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}) * Q_j)$

#### Decision Variable
\begin{equation}
  X_{g,j,t} = \left \{
  \begin{aligned}
    &1, && \text{if team g starts working on engine j at day t}\  \\
    &0, && \text{otherwise}
  \end{aligned} \right.
\end{equation} 

#### Constraints 
##### Maintenance of engine j can be performed at most once during the planning period

$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}\leq 1$ for all $j \in M$

##### Maintaince must be completed within the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t} * (t + \mu_{g,j} -1) \leq T$ for all $j \in M$

##### Teams can only work on one engine at a time
$\sum_{j=1}^{M} X_{g,j,t_a} * (\sum_{t=t_a}^{min \{ T,t + \mu_{g,j} -1\}} \sum_{j=1}^{M} X_{g,j,t_a}) \leq 1$ for all $g \in G, t_a \in T$

### 1. Linear model

In [8]:
import pandas as pd

In [14]:
df = pd.read_csv('../data/RUL_consultancy_predictions_A3.csv', sep=';', index_col='id')

In [33]:
M = list(df.index)

G = {1: 'A',
     2: 'A',
     3: 'B',
     4: 'B'}

c = {i:5 for i in range(1,21)}
c.update({i:7 for i in range(21,41)})
c.update({i:9 for i in range(41,61)})
c.update({i:5 for i in range(61,81)})
c.update({i:3 for i in range(81,101)})

mu_a = {i:4 for i in range(1,26)}
mu_a.update({i:6 for i in range(26,51)})
mu_a.update({i:3 for i in range(51,76)})
mu_a.update({i:5 for i in range(76,101)})
mu_b = {i:mu_a[i]+1 for i in range(1,34)}
mu_b.update({i:mu_a[i]+2 for i in range(34,68)})
mu_b.update({i:mu_a[i]+1 for i in range(68,101)})

mu = {g: {j: mu_a[j] if typ=='A' else mu_b[j] for j in M} for g, typ in G.items()}

T = 25

In [54]:
def get_penalty(g, j, t):
    '''
    Calculates penalty if team g starts working on engine j at day t
    '''
    RUL = df.loc[j].values[0]
    c_per_day = c[j]
    nr_days_costs = t - RUL + mu[g][j] - 1

    if nr_days_costs > 0:
        costs = nr_days_costs * c_per_day
        return costs
    else:
        return 0

In [55]:
P = {g:{j:{t: get_penalty(g,j,t) for t in range(1, T+1)} for j in M} for g in G}